In [1]:
import numpy as np
import pandas as pd

result = pd.read_csv('sample/results.csv', encoding='cp949')
result

,파일명,선종,상태,위치
0,1.jpg,범장망,표류,LAT N33.22.872 LON E124.5.074
1,2.jpg,범장망,이동,LAT N32.48.241 LON E124.38.553
2,3.jpg,범장망,조업,LAT N32.55.893 LON E124.33.731
3,4.jpg,타망,이동,LAT N35.40.891 LON E124.25.481
4,5.jpg,타망,이동,LAT N32.23.668 LON E125.33.053
5,6.jpg,타망,이동,LAT N32.59.352 LON E124.29.609
6,7.jpg,유망,조업,LAT N34.54.232 LON E124.42.394
7,8.jpg,유망,조업,LAT N34.41.829 LON E124.20.120
8,9.jpg,유망,표류,LAT N36.56.279 LON E124.39.427
9,10.jpg,안강망,조업,LAT N34.15.543 LON E125.41.175


In [2]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

# 사전 학습된 ResNet50 모델 로드 (ImageNet 데이터셋으로 학습됨)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 마지막 레이어를 교체해 새로운 모델 구성
x = base_model.output
x = GlobalAveragePooling2D()(x)  # 전역 평균 풀링
x = Dense(1024, activation='relu')(x)  # 완전 연결 레이어
predictions = Dense(2, activation='softmax')(x)  # 클래스 개수에 맞춘 출력 레이어

# 새로운 모델 정의
model = Model(inputs=base_model.input, outputs=predictions)

# 사전 학습된 레이어는 학습되지 않도록 고정
for layer in base_model.layers:
    layer.trainable = False

# 모델 컴파일
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# 데이터셋 전처리 (ImageDataGenerator를 사용하여 이미지 증식)
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
train_generator = train_datagen.flow_from_directory(
    'sample', target_size=(224, 224), batch_size=32, class_mode='categorical'
)

# 모델 학습
model.fit(train_generator, steps_per_epoch=100, epochs=10)

# 학습된 모델 저장  
model.save('resnet50_finetuned.h5')


Found 12 images belonging to 2 classes.
Epoch 1/10


c:\Users\kangdongwoo\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


100/100 ━━━━━━━━━━━━━━━━━━━━ 6s 364us/step - accuracy: 0.5000 - loss: 0.6998
Epoch 2/10


c:\Users\kangdongwoo\AppData\Local\Programs\Python\Python310\lib\contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 71us/step - accuracy: 0.5000 - loss: 2.8722  
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 81us/step - accuracy: 0.5000 - loss: 1.2584  
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 71us/step - accuracy: 0.5000 - loss: 0.9755  
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 71us/step - accuracy: 0.5000 - loss: 1.5544  
Epoch 6/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 81us/step - accuracy: 0.5000 - loss: 1.3702  
Epoch 7/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 91us/step - accuracy: 0.5000 - loss: 0.7739  
Epoch 8/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 81us/step - accuracy: 0.5000 - loss: 0.7344  
Epoch 9/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 81us/step - accuracy: 0.5000 - loss: 1.0873  
Epoch 10/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 71us/step - accuracy: 0.5000 - loss: 1.0805  


In [3]:
import folium

# 좌표 데이터 (위도, 경도)
locations = [
    {"name": "차", "lat": 32 + 11/60, "lon": 125 + 25/60},
    {"name": "카", "lat": 33 + 20/60, "lon": 124 + 8/60},
    {"name": "타", "lat": 34 + 0/60, "lon": 124 + 0/60 + 30/3600},
    {"name": "파", "lat": 35 + 0/60, "lon": 124 + 7/60 + 30/3600},
    {"name": "하", "lat": 35 + 30/60, "lon": 124 + 30/60},
    {"name": "거", "lat": 36 + 45/60, "lon": 124 + 30/60},
    {"name": "너", "lat": 37 + 0/60, "lon": 124 + 20/60},
    {"name": "바", "lat": 32 + 11/60, "lon": 126 + 45/60},
]

# 초기 지도의 중심 좌표 (평균 값 사용)
center_lat = sum([loc["lat"] for loc in locations]) / len(locations)
center_lon = sum([loc["lon"] for loc in locations]) / len(locations)

# 지도 생성
mymap = folium.Map(location=[center_lat, center_lon], zoom_start=7)

# 각 위치에 마커 추가
for loc in locations:
    folium.Marker(
        location=[loc["lat"], loc["lon"]],
        popup=loc["name"]
    ).add_to(mymap)

# 지도 저장
mymap.save("map.html")

# 지도를 출력하고 싶다면, 주피터 노트북 환경에서는 다음 코드로 출력 가능
mymap


In [4]:
pip install easyocr

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import cv2
import pytesseract
import pandas as pd
import os
from PIL import Image

# Tesseract 경로 설정 (Windows 사용자만 필요)
# p ytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# 이미지 불러오기
img_path = 'sample/class2/9.jpg'
image = cv2.imread(img_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
pil_image = Image.fromarray(image_rgb)

# OCR을 이용한 텍스트 추출
text = pytesseract.image_to_string(pil_image, lang='eng')  # lang='kor'으로 한국어도 가능

# 추출된 텍스트 출력
print("Extracted Text: \n", text)

# 추출된 텍스트를 데이터프레임으로 변환
# 여기서는 예시로, 각 줄을 행으로 취급
lines = text.split('\n')
data = [line.split() for line in lines if line.strip()]  # 빈 줄 제거 및 단어로 분할
df = pd.DataFrame(data)

# 데이터프레임 출력
df

Extracted Text: 
 


""


In [7]:
import pandas as pd
import re
from shapely.geometry import Point, Polygon

# 좌표 변환 함수 (DMS -> Decimal)
def dms_to_decimal(degrees, minutes_seconds):
    minutes = float(minutes_seconds) / 60
    return degrees + minutes

# 좌표를 추출하는 함수 (데이터프레임 행에서)
def extract_coordinates(row):
    print(row)
    # 위도 추출
    lat_match = re.match(r"(\d+)°\s*(\d+\.\d+)\"", row['2'] + ' ' + row['3'])
    lat_degrees = int(lat_match.group(1))
    lat_minutes_seconds = float(lat_match.group(2))

    # 경도 추출
    lon_match = re.match(r"(\d+)°\s*(\d+\.\d+)", row['6'] + ' ' + row['7'])
    lon_degrees = int(lon_match.group(1))
    lon_minutes_seconds = float(lon_match.group(2))

    # 위도와 경도를 소수점 형식으로 변환
    latitude = dms_to_decimal(lat_degrees, lat_minutes_seconds)
    longitude = dms_to_decimal(lon_degrees, lon_minutes_seconds)

    return latitude, longitude

# 예시 데이터프레임 (주어진 형식에 맞춤)
# data = {
#     '0': ['TLat', 'It'],
#     '1': ['N', 'Of'],
#     '2': ['32°', 'S'],
#     '3': ['55.893"', 'Rng:'],
#     '4': ['TLon', '353m'],
#     '5': ['E', 'Ins'],
#     '6': ['124°', 'Nav'],
#     '7': ['33.731', 'HdgUncert=0.13']
# }

# df = pd.DataFrame(data)

# 위도와 경도 추출
latitude, longitude = extract_coordinates(df.loc[3])

print(f"변환된 위도: {latitude}")
print(f"변환된 경도: {longitude}")

df

KeyError: 3

In [8]:
import re
from shapely.geometry import Point, Polygon

# 좌표 변환 함수 (DMS -> Decimal)
def dms_to_decimal(degrees, minutes_seconds):
    minutes = float(minutes_seconds) / 60
    return degrees + minutes

# 좌표를 추출하는 함수 (데이터프레임의 특정 형식에서)
def extract_coordinates(df):
    # 위도 N 32° 55.893"
    lat_degree = int(df['TLat'][1][:-1])  # 위도는 N이므로 숫자만 추출 (32)
    lat_minutes_seconds = float(df['TLat'][2][:-1])  # 55.893"에서 숫자만 추출

    # 경도 E 124° 33.731"
    lon_degree = int(df['TLon'][1][:-1])  # 경도는 E이므로 숫자만 추출 (124)
    lon_minutes_seconds = float(df['TLon'][2][:-1])  # 33.731"에서 숫자만 추출

    # 위도와 경도를 소수점 형식으로 변환
    latitude = dms_to_decimal(lat_degree, lat_minutes_seconds)
    longitude = dms_to_decimal(lon_degree, lon_minutes_seconds)

    return latitude, longitude

# 예시 데이터 (입력 형식에 맞게)
data = {
    'TLat': ['N', '32°', '55.893"'],
    'TLon': ['E', '124°', '33.731"']
}

# 위도와 경도 추출
latitude, longitude = extract_coordinates(data)

print(f"변환된 위도: {latitude}")
print(f"변환된 경도: {longitude}")

# 한중 잠정조치수역 동쪽 경계선 좌표 (위도, 경도)
boundary_coords = [
    (37 + 00/60, 124 + 20/60),  # 너 - A16
    (36 + 45/60, 124 + 30/60),  # 거 - A15
    (35 + 30/60, 124 + 30/60),  # 하 - A14
    (35 + 00/60, 124 + 7.5/60), # 파 - A13
    (34 + 00/60, 124 + 0.5/60), # 타 - A12
    (33 + 20/60, 124 + 8/60),   # 카 - A11
    (32 + 11/60, 125 + 25/60),  # 차 - A10
    (32 + 11/60, 126 + 45/60),  # 바 - K6
    (32 + 40/60, 127 + 0/60)    # 사 - K7
]

# 경계선을 폴리곤으로 정의
boundary_polygon = Polygon(boundary_coords)

# 선박의 위치를 변환된 좌표로 정의
ship_location = Point(latitude, longitude)

# 선박이 경계선 안에 있는지 확인
if boundary_polygon.contains(ship_location):
    print("선박이 한국 어선 구역에 진입하지 않았습니다.")
else:
    print("선박이 한국 어선 구역에 진입했습니다.")


변환된 위도: 32.93155
변환된 경도: 124.56218333333334
선박이 한국 어선 구역에 진입했습니다.


In [9]:
import pytesseract
from PIL import Image

# Tesseract 경로 설정 (환경에 맞게 설정 필요)
pytesseract.pytesseract.tesseract_cmd = r'C:\Users\kangdongwoo\AppData\Local\Programs\Python\Python310\tesseract.exe'

def extract_bottom_text(image_path, height_ratio=0.2):
    """
    이미지 하단 부분의 텍스트만 추출하는 함수
    :param image_path: 이미지 경로
    :param height_ratio: 하단 부분의 비율 (예: 0.2는 하단 20%에 해당하는 영역)
    :return: 하단 부분에서 추출한 텍스트
    """
    # 이미지 열기
    image = Image.open(image_path)
    width, height = image.size

    # Tesseract의 image_to_data를 사용하여 각 단어의 위치와 함께 텍스트 추출
    ocr_data = pytesseract.image_to_data(image, output_type=pytesseract.Output.DICT)

    # 하단 부분의 시작 Y 좌표 설정 (비율로 하단 20% 영역 지정)
    bottom_start_y = int(height * (1 - height_ratio))

    # 하단 부분의 텍스트만 추출
    bottom_text = []
    for i, text in enumerate(ocr_data['text']):
        # 각 단어의 위치 좌표 (left, top, width, height)
        top = ocr_data['top'][i]
        if text.strip() and top >= bottom_start_y:  # 하단 부분에 해당하는 단어만 추출
            bottom_text.append(text)

    return ' '.join(bottom_text)

# 이미지 파일 경로
image_path = 'sample/class2/10.jpg'

# 이미지의 하단 20% 영역에서 텍스트 추출
extracted_text = extract_bottom_text(image_path, height_ratio=0.2)

print("하단 부분에서 추출된 텍스트:", extracted_text)


하단 부분에서 추출된 텍스트: 


In [14]:
import easyocr

# EasyOCR Reader 초기화 (한국어 및 영어 설정 가능)
reader = easyocr.Reader(['ko', 'en'])

# 이미지 경로
image_path = 'sample/class2/8.jpg'

# 텍스트 추출
result = reader.readtext(image_path)

# 결과 출력
for (bbox, text, prob) in result:
    print(f"Detected Text: {text}, Confidence: {prob}")


Detected Text: InAIlin, Confidence: 0.030435361788865883
Detected Text: 익  rilsr|, Confidence: 0.003985708339073854
Detected Text: 미 ShRna, Confidence: 0.11211563254218385
Detected Text: ZUUUTI, Confidence: 0.18189194352039892
